In [16]:
import pandas as pd
import numpy as np
import gspread
from datetime import date
from datetime import datetime
from datetime import datetime, timedelta
from pandas import json_normalize

import re

import sys                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
sys.path.append('../')

import zoho_crm_api_module as crm

import sys
sys.path.append('../../../zoho_data_app/')

# importing
import main_module as m

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
gc = gspread.oauth(
    credentials_filename= r'C:\Users\fajar.fatoni\Documents\Python\Data\Google Credentials\karma-oauth.json'
)

SHEET_ID = '1_zOrWK1gAo-tHDsKqU6AaqVzi4xaNpOELDwtbBx5F6M'
SHEET_NAME = 'Karma Beach - LIVE'


cols = [
    "id","Created_Time", "First_Name", "Last_Name", "Email", "Birthdate", "Phone", 
    "Street", "City", "Country", "Brand", "Lead_Sub_Brand", "Lead_Source", 
    "Lead_Source_Description", "Lead_Locations", "Lead_History", "Website", 
    "Card_Number", "Card_Expired", "Do_you_live_in_Bali"
]

live_cols = [
    "Record Id", "Created Time", "First Name", "Last Name", "Email", "Birthdate", 
    "Phone", "Street", "City", "Country", "Lead Brand", "Lead Sub-Brand", 
    "Lead Source", "Lead Source Description", "Lead Locations", "Lead History", 
    "Website", "Card Number", "Card Expired", "Do you live in Bali?"
]

# Function to fix the encoding
def fix_encoding(garbled_string):
    try:
        # Encode the garbled string to bytes using 'latin1' and then decode it to 'utf-8'
        return garbled_string.encode('latin1').decode('utf-8')
    except Exception as e:
        return garbled_string  # Return the original string if there's an error


In [18]:
crm_token = crm.get_crm_token()

In [19]:
df = crm.get_kbb_vip_data(crm_token, cols)
df.fillna('empty', inplace=True)
new_df = crm.get_kbb_vip_filter(df)
new_df['id'] = 'zcrm_'+new_df['id']
new_df.rename(columns={'id': 'Record Id'}, inplace=True)
new_df['Created_Time'] = new_df['Created_Time'].str.replace('T', ' ').str.replace(r'\+\d{2}:\d{2}', '', regex=True)
new_df['Created_Time']  = pd.to_datetime(new_df['Created_Time']).dt.strftime('%d/%m/%Y')
new_df['Card_Expired']  = pd.to_datetime(new_df['Card_Expired']).dt.strftime('%d/%m/%Y')
new_df['First_Name'] = new_df['First_Name'].apply(fix_encoding)
new_df['Last_Name'] = new_df['Last_Name'].apply(fix_encoding)
m.clean_number(new_df, 'Phone')
m.replace_not_valid_to_empty(new_df, 'Phone')
# new_df = df.copy()
new_df.sort_values('Created_Time', ascending=False, inplace=True)
new_df.reset_index(drop=True, inplace=True)
count_data_awal = new_df.shape[0]
new_df

c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\zoho_api\sync_module\..\zoho_crm_api_module.py:221: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  concat_df['Lead_Locations'].replace('\[|\]|\'','', regex=True, inplace=True)
c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\zoho_api\sync_module\../../../zoho_data_app\main_module.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pand

,Record Id,Created_Time,First_Name,Last_Name,Email,Birthdate,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website,Card_Number,Card_Expired,Do_you_live_in_Bali
0,zcrm_3584690000533708074,22/03/2025,Julia,Babakova,missjuly13031994@gmail.com,empty,79508637032,empty,empty,Russia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300220003,22/03/2026,False
1,zcrm_3584690000533706001,22/03/2025,Michelle,Bartlett,michelle.bartlett01@gmail.com,empty,6421817352,empty,empty,New Zealand,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300220001,22/03/2026,False
2,zcrm_3584690000533607081,22/03/2025,Andreas,Triandafilidis,dray.nz@gmail.com,empty,64273331631,empty,empty,empty,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300220002,22/03/2026,False
3,zcrm_3584690000533315001,21/03/2025,KEVIN,HEATRICK,mareetha@tpg.com.au,empty,61429095232,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300210005,21/03/2026,False
4,zcrm_3584690000533310001,21/03/2025,Stepan,Shatkovski,pints_jumbles7t@icloud.com,empty,6281227843993,empty,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300210004,21/03/2026,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,zcrm_3584690000529185033,14/03/2025,Skai,Bareckaite,skaiste124@gmail.com,empty,6281325169510,empty,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300140019,14/03/2026,False
105,zcrm_3584690000529186033,14/03/2025,Migunova,Anastasia,laomigunova@gmail.com,empty,6282339514652,empty,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300140021,14/03/2026,False
106,zcrm_3584690000529187001,14/03/2025,Lucy,Mackinnon,lucy.mackinnon@hotmail.co.uk,empty,447411845405,empty,empty,United Kingdom,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300140017,14/03/2026,False
107,zcrm_3584690000529254001,14/03/2025,Mikhailo,Volochai,volochay.ms@gmail.com,empty,380669558920,empty,empty,Ukraine,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300140018,14/03/2026,False


In [20]:
new_df['First_Name'] = new_df['First_Name'].str.title()
new_df['Last_Name'] = new_df['Last_Name'].str.title()
new_df['Email'] = m.lowercase(new_df, 'Email')
m.clean_space(new_df, 'Email')

new_df
# new_df.to_excel(r"C:\Users\fajar.fatoni\Documents\Python\Data\kbbvip_live.xlsx", index=False)

c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\zoho_api\sync_module\../../../zoho_data_app\main_module.py:94: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


,Record Id,Created_Time,First_Name,Last_Name,Email,Birthdate,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website,Card_Number,Card_Expired,Do_you_live_in_Bali
0,zcrm_3584690000533708074,22/03/2025,Julia,Babakova,missjuly13031994@gmail.com,empty,79508637032,empty,empty,Russia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300220003,22/03/2026,False
1,zcrm_3584690000533706001,22/03/2025,Michelle,Bartlett,michelle.bartlett01@gmail.com,empty,6421817352,empty,empty,New Zealand,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300220001,22/03/2026,False
2,zcrm_3584690000533607081,22/03/2025,Andreas,Triandafilidis,dray.nz@gmail.com,empty,64273331631,empty,empty,empty,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300220002,22/03/2026,False
3,zcrm_3584690000533315001,21/03/2025,Kevin,Heatrick,mareetha@tpg.com.au,empty,61429095232,empty,empty,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300210005,21/03/2026,False
4,zcrm_3584690000533310001,21/03/2025,Stepan,Shatkovski,pints_jumbles7t@icloud.com,empty,6281227843993,empty,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300210004,21/03/2026,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,zcrm_3584690000529185033,14/03/2025,Skai,Bareckaite,skaiste124@gmail.com,empty,6281325169510,empty,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300140019,14/03/2026,False
105,zcrm_3584690000529186033,14/03/2025,Migunova,Anastasia,laomigunova@gmail.com,empty,6282339514652,empty,empty,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300140021,14/03/2026,False
106,zcrm_3584690000529187001,14/03/2025,Lucy,Mackinnon,lucy.mackinnon@hotmail.co.uk,empty,447411845405,empty,empty,United Kingdom,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300140017,14/03/2026,False
107,zcrm_3584690000529254001,14/03/2025,Mikhailo,Volochai,volochay.ms@gmail.com,empty,380669558920,empty,empty,Ukraine,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,empty,empty,https://karmagroup.com/karma-beach-vip/,2025000300140018,14/03/2026,False


In [21]:
# # mencari tanggal 8 hari kebelakang
# calculate_date = datetime.now() - timedelta(days=8)
# new_cal_date = str(calculate_date.strftime("%Y-%m-%d")).split(' ')
# first_date = new_cal_date[0]

# # mencari tanggal hari ini
# today = datetime.now() 
# second_date = today.strftime("%Y-%m-%d")

# print(first_date)
# print(second_date)

In [22]:
new_df.replace('empty', '', inplace=True)
new_df

,Record Id,Created_Time,First_Name,Last_Name,Email,Birthdate,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website,Card_Number,Card_Expired,Do_you_live_in_Bali
0,zcrm_3584690000533708074,22/03/2025,Julia,Babakova,missjuly13031994@gmail.com,,79508637032,,,Russia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300220003,22/03/2026,False
1,zcrm_3584690000533706001,22/03/2025,Michelle,Bartlett,michelle.bartlett01@gmail.com,,6421817352,,,New Zealand,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300220001,22/03/2026,False
2,zcrm_3584690000533607081,22/03/2025,Andreas,Triandafilidis,dray.nz@gmail.com,,64273331631,,,,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300220002,22/03/2026,False
3,zcrm_3584690000533315001,21/03/2025,Kevin,Heatrick,mareetha@tpg.com.au,,61429095232,,,Australia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300210005,21/03/2026,False
4,zcrm_3584690000533310001,21/03/2025,Stepan,Shatkovski,pints_jumbles7t@icloud.com,,6281227843993,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300210004,21/03/2026,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,zcrm_3584690000529185033,14/03/2025,Skai,Bareckaite,skaiste124@gmail.com,,6281325169510,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300140019,14/03/2026,False
105,zcrm_3584690000529186033,14/03/2025,Migunova,Anastasia,laomigunova@gmail.com,,6282339514652,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300140021,14/03/2026,False
106,zcrm_3584690000529187001,14/03/2025,Lucy,Mackinnon,lucy.mackinnon@hotmail.co.uk,,447411845405,,,United Kingdom,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300140017,14/03/2026,False
107,zcrm_3584690000529254001,14/03/2025,Mikhailo,Volochai,volochay.ms@gmail.com,,380669558920,,,Ukraine,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300140018,14/03/2026,False


In [23]:
# Get data from Google Sheets
spreadsheet = gc.open_by_key(SHEET_ID)
worksheet = spreadsheet.worksheet(SHEET_NAME)
rows = worksheet.get_all_records()

# Create DataFrame from the rows
df_live = pd.DataFrame(rows)

# Convert 'Card Expired' to datetime format, allowing for mixed formats
df_live['Card Expired'] = pd.to_datetime(df_live['Card Expired'], dayfirst=True)

# Format 'Card Expired' back to the desired string format
df_live['Card Expired'] = df_live['Card Expired'].dt.strftime('%d/%m/%Y')
df_live['First Name'] = df_live['First Name'].apply(fix_encoding)
df_live['Last Name	'] = df_live['Last Name'].apply(fix_encoding)
# Display the DataFrame
df_live

,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,...,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?,Last Name\t
0,zcrm_3584690000533708074,22/03/2025,Julia,Babakova,missjuly13031994@gmail.com,,79508637032,,,Russia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300220003,22/03/2026,FALSE,Babakova
1,zcrm_3584690000533607081,22/03/2025,Andreas,Triandafilidis,dray.nz@gmail.com,,64273331631,,,,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300220002,22/03/2026,FALSE,Triandafilidis
2,zcrm_3584690000533706001,22/03/2025,Michelle,Bartlett,michelle.bartlett01@gmail.com,,6421817352,,,New Zealand,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300220001,22/03/2026,FALSE,Bartlett
3,zcrm_3584690000533315001,21/03/2025,Kevin,Heatrick,mareetha@tpg.com.au,,61429095232,,,Australia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300210005,21/03/2026,FALSE,Heatrick
4,zcrm_3584690000533310001,21/03/2025,Stepan,Shatkovski,pints_jumbles7t@icloud.com,,6281227843993,,,Indonesia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300210004,21/03/2026,FALSE,Shatkovski
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9297,zcrm_3584690000005983059,02/04/2019,Ken,Hutchins,ken@gusto.com.au,18/03/1966,61414949924,3215 /101 Bathurst st,Sydney,Australia,...,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Kandara,"Other - Past Guests, Karma Club - Membership, ...",https://karmagroup.com/karma-beach-vip/,2023001200260004,26/12/2024,FALSE,Hutchins
9298,zcrm_3584690000006011328,02/04/2019,Parth,J,parthsarthi@gmail.com,,6597767716,15 D Balmoral Road 10 - / 6,Singapore,Singapore,...,Other,Past Guests,Karma Beach VIP membership sign up 2023,Karma Kandara,Other - Past Guests,https://karmagroup.com/karma-beach-vip/,2024001200260006,26/12/2025,FALSE,J
9299,zcrm_3584690000007176939,02/04/2019,David,Gozali,david_gozali@hotmail.com,27/03/1993,6281289236128,Pantai Indah Kapuk,Jakarta,Indonesia,...,Karma Club,Membership,Karma Beach VIP membership sign up 2023,,"Karma Club - Membership, Other - Web Chat",https://karmagroup.com/karma-beach-vip/,2025000100310007,31/01/2026,FALSE,Gozali
9300,zcrm_3584690000007204035,02/04/2019,Sri,Ratna,ratna_pink@yahoo.com,24/06/1986,6597108106,06-438 PUNGGOL WALK,Singapore,Singapore,...,Karma Club,Membership,Karma Beach VIP membership sign up 2023,,Karma Club - Membership,https://karmagroup.com/karma-beach-vip/,2024001000250004,25/10/2025,FALSE,Ratna


In [24]:
df_update = new_df.loc[~new_df['Record Id'].isin(df_live['Record Id'])].copy()
df_update

,Record Id,Created_Time,First_Name,Last_Name,Email,Birthdate,Phone,Street,City,Country,Brand,Lead_Sub_Brand,Lead_Source,Lead_Source_Description,Lead_Locations,Lead_History,Website,Card_Number,Card_Expired,Do_you_live_in_Bali
41,zcrm_3584690000530880033,17/03/2025,Form,Test3,wayan.wirka+test03@karmagroup.com,,6281916143350,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300170005,17/03/2026,False
45,zcrm_3584690000530880001,17/03/2025,Form,Test,wayan.wirka+test02@karmagroup.com,,6281916143350,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300170003,17/03/2026,False
57,zcrm_3584690000530624074,17/03/2025,Ferry,Yudhitamatests,ferry.yudhitama+21233@karmagroup.com,,6287861085142,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300170004,17/03/2026,False


In [25]:
df_update.sort_values(by=['Created_Time'], inplace=True, ascending=False)
rename_mapping = dict(zip(cols, live_cols))
df_update = df_update.rename(columns=rename_mapping)
df_update.reset_index(drop=True, inplace=True)
df_update


,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,Lead Brand,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?
0,zcrm_3584690000530880033,17/03/2025,Form,Test3,wayan.wirka+test03@karmagroup.com,,6281916143350,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300170005,17/03/2026,False
1,zcrm_3584690000530880001,17/03/2025,Form,Test,wayan.wirka+test02@karmagroup.com,,6281916143350,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300170003,17/03/2026,False
2,zcrm_3584690000530624074,17/03/2025,Ferry,Yudhitamatests,ferry.yudhitama+21233@karmagroup.com,,6287861085142,,,Indonesia,Karma Resorts,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300170004,17/03/2026,False


In [26]:
# Concatenate DataFrames
df_final = pd.concat([df_update, df_live], ignore_index=True)

# Replace 'empty' with ''
df_final.replace('empty', '', inplace=True)

# Convert 'Do you live in Bali?' to uppercase
df_final['Do you live in Bali?'] = df_final['Do you live in Bali?'].astype(str).str.upper()

# Convert 'Created Time' to datetime format
df_final['Created Time'] = pd.to_datetime(df_final['Created Time'], dayfirst=True, errors='coerce')


# Sort by 'Created Time' in descending order
df_final.sort_values('Created Time', ascending=False, inplace=True)

# Format 'Created Time' back to the desired string format
df_final['Created Time'] = df_final['Created Time'].dt.strftime('%d/%m/%Y')

df_final

,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,...,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?,Last Name\t
3,zcrm_3584690000533708074,22/03/2025,Julia,Babakova,missjuly13031994@gmail.com,,79508637032,,,Russia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300220003,22/03/2026,FALSE,Babakova
4,zcrm_3584690000533607081,22/03/2025,Andreas,Triandafilidis,dray.nz@gmail.com,,64273331631,,,,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300220002,22/03/2026,FALSE,Triandafilidis
5,zcrm_3584690000533706001,22/03/2025,Michelle,Bartlett,michelle.bartlett01@gmail.com,,6421817352,,,New Zealand,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300220001,22/03/2026,FALSE,Bartlett
6,zcrm_3584690000533315001,21/03/2025,Kevin,Heatrick,mareetha@tpg.com.au,,61429095232,,,Australia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300210005,21/03/2026,FALSE,Heatrick
7,zcrm_3584690000533310001,21/03/2025,Stepan,Shatkovski,pints_jumbles7t@icloud.com,,6281227843993,,,Indonesia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300210004,21/03/2026,FALSE,Shatkovski
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9281,zcrm_3584690000008001711,02/04/2019,Dawn,Herring,dawn.herring@gmail.com,,6281338527802,,,Indonesia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000300160004,16/03/2025,FALSE,Herring
9280,zcrm_3584690000007896028,02/04/2019,Wesley,Powell,wez.powell@gmail.com,,61459554763,,,Australia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000500240018,24/05/2025,FALSE,Powell
9279,zcrm_3584690000007256953,02/04/2019,Ake,Lal Ponini,lalponini.larissa@gmail.com,30/05/1990,61422226902,311/8 GARFIELD ST,Richmond,Australia,...,Karma Club,Membership,Karma Beach VIP membership sign up 2023,,Karma Club - Membership,https://karmagroup.com/karma-beach-vip/,2024000200200002,20/02/2025,FALSE,Lal Ponini
9278,zcrm_3584690000007239601,02/04/2019,Jem,Marcelo,jemarelalarmarcelo@gmail.com,10/06/1990,60183938375,Symphony Hill,Cyberjaya,Malaysia,...,Karma Club,Membership,Karma Beach VIP membership sign up 2023,,Karma Club - Membership,https://karmagroup.com/karma-beach-vip/,2024000200100008,10/02/2025,FALSE,Marcelo


In [27]:
df_live.columns
# new_df.columns

Index(['Record Id', 'Created Time', 'First Name', 'Last Name', 'Email',
       'Birthdate', 'Phone', 'Street', 'City', 'Country', 'Lead Brand',
       'Lead Sub-Brand', 'Lead Source', 'Lead Source Description',
       'Lead Locations', 'Lead History', 'Website', 'Card Number',
       'Card Expired', 'Do you live in Bali?', 'Last Name\t'],
      dtype='object')

In [28]:
# Function to sanitize DataFrame
def sanitize_dataframe(df):
    # Replace invalid float values with None
    df = df.applymap(lambda x: None if isinstance(x, float) and (np.isinf(x) or np.isnan(x)) else x)
    return df

# Sanitize the DataFrame
df_final = sanitize_dataframe(df_final)
df_final['Card Number'] = df_final['Card Number'].astype(str)
df_final.drop(df_final.loc[df_final['Email'].str.contains('karmagroup.com', regex=True, flags=re.I)].index, inplace=True)
df_final

C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_17248\4224994593.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: None if isinstance(x, float) and (np.isinf(x) or np.isnan(x)) else x)


,Record Id,Created Time,First Name,Last Name,Email,Birthdate,Phone,Street,City,Country,...,Lead Sub-Brand,Lead Source,Lead Source Description,Lead Locations,Lead History,Website,Card Number,Card Expired,Do you live in Bali?,Last Name\t
3,zcrm_3584690000533708074,22/03/2025,Julia,Babakova,missjuly13031994@gmail.com,,79508637032,,,Russia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300220003,22/03/2026,FALSE,Babakova
4,zcrm_3584690000533607081,22/03/2025,Andreas,Triandafilidis,dray.nz@gmail.com,,64273331631,,,,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300220002,22/03/2026,FALSE,Triandafilidis
5,zcrm_3584690000533706001,22/03/2025,Michelle,Bartlett,michelle.bartlett01@gmail.com,,6421817352,,,New Zealand,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300220001,22/03/2026,FALSE,Bartlett
6,zcrm_3584690000533315001,21/03/2025,Kevin,Heatrick,mareetha@tpg.com.au,,61429095232,,,Australia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300210005,21/03/2026,FALSE,Heatrick
7,zcrm_3584690000533310001,21/03/2025,Stepan,Shatkovski,pints_jumbles7t@icloud.com,,6281227843993,,,Indonesia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,,https://karmagroup.com/karma-beach-vip/,2025000300210004,21/03/2026,FALSE,Shatkovski
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9281,zcrm_3584690000008001711,02/04/2019,Dawn,Herring,dawn.herring@gmail.com,,6281338527802,,,Indonesia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000300160004,16/03/2025,FALSE,Herring
9280,zcrm_3584690000007896028,02/04/2019,Wesley,Powell,wez.powell@gmail.com,,61459554763,,,Australia,...,Karma Beach,Karma Beach VIP 2023,Karma Beach VIP membership sign up 2023,,Other - Other,https://karmagroup.com/karma-beach-vip/,2024000500240018,24/05/2025,FALSE,Powell
9279,zcrm_3584690000007256953,02/04/2019,Ake,Lal Ponini,lalponini.larissa@gmail.com,30/05/1990,61422226902,311/8 GARFIELD ST,Richmond,Australia,...,Karma Club,Membership,Karma Beach VIP membership sign up 2023,,Karma Club - Membership,https://karmagroup.com/karma-beach-vip/,2024000200200002,20/02/2025,FALSE,Lal Ponini
9278,zcrm_3584690000007239601,02/04/2019,Jem,Marcelo,jemarelalarmarcelo@gmail.com,10/06/1990,60183938375,Symphony Hill,Cyberjaya,Malaysia,...,Karma Club,Membership,Karma Beach VIP membership sign up 2023,,Karma Club - Membership,https://karmagroup.com/karma-beach-vip/,2024000200100008,10/02/2025,FALSE,Marcelo


In [29]:
# Update keseluruhan data, delete data yang lama terlebih dahulu
# clear all row before update
worksheet.clear()
worksheet.update([df_final.columns.values.tolist()] + df_final.values.tolist())

{'spreadsheetId': '1_zOrWK1gAo-tHDsKqU6AaqVzi4xaNpOELDwtbBx5F6M',
 'updatedRange': "'Karma Beach - LIVE'!A1:U9303",
 'updatedRows': 9303,
 'updatedColumns': 21,
 'updatedCells': 195363}